# Example 2: 

Let us suppose we want to make a 3D model that is layered in one direction and also has a blob in it.

There are two ways you could do this in axisem3D:
1) Create a homogenous 1D model and then a layered 3D model + a blob (this is what is happening in this example)

2) Use a layered 1D model and then just a blob in a homogenous 3D model - this is definitely the best way to do it. In
this case I am just doing this example to demonstrate that you can have whatever 3D model you want and inject blobs
into it

In [1]:
# Importing what we need: 
import numpy as np
from model_class import model
from gen_sphere import gen_sphere
from write_NetCDF import writeNetCDF
from add_sphere import addSphere

Okay, lets make a model object that is originally empty. This is a model with dimensions of 20 km x 20 km x 50 km: 

In [2]:
# Define our model parameters
x = np.array([0, 20000]) 
y = np.array([0, 20000]) 
z = np.array([0, 50000]) 
freq = 3        # [Hz]
min_vel = 3000  # [m/s]
epw = 3         # 3 elements per wavelength - pretty high res.

# Instantiate our model object
m = model(x,y,z, epw, freq, min_vel, oversaturation=1)

So now we might want to put some layers into our model, or some more complex structure. As I mentioned above, I would not reccomend doing a layered structure using this method in axisem, instead you should use a layered 1D .bm model and then have a 3D model that is mostly homogenous. However I want to demonstrate here that the 3D model can be whatever you want it to be 

In [3]:
# Lets take our initial background model for vp:
vp_array = m.bm_vp

dis = np.array([0, 10000, 18000, 30000, 50000]) # Define some random discontinuities 
layer_vps = np.array([3000, 3300, 4000, 4400]) # Defining some new velocity for these layers 

# get corresponding z indicies for the discontinuities:
dis_indices = dis*m.nz/(z[1]-z[0])

for i in range(len(layer_vps)):
    vp_array[:,:, int(dis_indices[i]):int(dis_indices[i+1])] = layer_vps[i]

# Now we have our layered array, we can update the bm_models - lets imagine that the vs structure is the same:
# But we dont change the rho structure so it is still homogenous - this is obviously kinda silly but still...
m.set_bm_vp(vp_array)
m.set_bm_vs(vp_array)


Okay, now we can write this layered 3D model to a file if we want: 

In [4]:
writeNetCDF(m, "example2.nc")

Data written to file  example2.nc


If we were to run this and view it in paraview we would get the following: 
<img src="example_figs/Example2_noblob.png" width="400">

Now we might want to add a blob in there somewhere...

In [ ]:

# What if we now want to add in a blobs to this? Simple. We first take our model and generate a sphere:
sph = gen_sphere(model=m, radius=5000, RHO=2500, VP=2500, VS=2300)

sphere_centre = [5000, 10000, 49400]

sph.set_centre(sphere_centre, m, print_conf='n')
# Add sphere to model:
m = addSphere(sph, m)

writeNetCDF(m, "example2_withblob.nc")


In this case we get the following: 
<img src="example_figs/Example2_blob.png" width="400">